In [ ]:
import os
from ciao_contrib.runtool import *
import glob
import pycrates as pcr
from paramio import *
from crates_contrib.utils import *
import numpy as np

objList = sorted(glob.glob("NGC*"))


for obj in objList[::]:
    print('RUNNING NGC{}'.format(obj))
    numObs = glob.glob('{}/*/repro'.format(obj))
    #print(len(numObs))
    sf = pcr.read_file('{}/sources/srcflux/srcflux_0.3-1.2.flux'.format(obj))
    mf = pcr.read_file('{}/sources/srcflux/srcflux_medium.flux'.format(obj))
    hf = pcr.read_file('{}/sources/srcflux/srcflux_hard.flux'.format(obj))
    bf = pcr.read_file('{}/sources/srcflux/srcflux_0.3-7.flux'.format(obj))
    if len(numObs) > 1:
        softRate = pcr.get_colvals(sf, 'MERGED_NET_RATE_APER')
        mediumRate = pcr.get_colvals(mf, 'MERGED_NET_RATE_APER')
        hardRate = pcr.get_colvals(hf, 'MERGED_NET_RATE_APER')
        broadRate = pcr.get_colvals(bf, 'MERGED_NET_RATE_APER')
    else:
        softRate = pcr.get_colvals(sf, 'NET_RATE_APER')
        mediumRate = pcr.get_colvals(mf, 'NET_RATE_APER')
        hardRate = pcr.get_colvals(hf, 'NET_RATE_APER')
        broadRate = pcr.get_colvals(bf, 'NET_RATE_APER')
    if len(numObs) > 1:
        softTime = pcr.get_colvals(sf, 'PSF_WEIGHTED_TOTAL_EXPOSURE_TIME')
        mediumTime = pcr.get_colvals(mf, 'PSF_WEIGHTED_TOTAL_EXPOSURE_TIME')
        hardTime = pcr.get_colvals(hf, 'PSF_WEIGHTED_TOTAL_EXPOSURE_TIME')
        broadTime = pcr.get_colvals(bf, 'PSF_WEIGHTED_TOTAL_EXPOSURE_TIME')
    else:
        softTime = pcr.get_colvals(sf, 'EXPOSURE')
        mediumTime = pcr.get_colvals(mf, 'EXPOSURE')
        hardTime = pcr.get_colvals(hf, 'EXPOSURE')
        broadTime = pcr.get_colvals(bf, 'EXPOSURE')
    if len(numObs) > 1:
        softHi = pcr.get_colvals(sf, 'MERGED_NET_RATE_APER_HI')
        softLo = pcr.get_colvals(sf, 'MERGED_NET_RATE_APER_LO')
        mediumHi = pcr.get_colvals(mf, 'MERGED_NET_RATE_APER_HI')
        mediumLo = pcr.get_colvals(mf, 'MERGED_NET_RATE_APER_LO')
        hardHi = pcr.get_colvals(hf, 'MERGED_NET_RATE_APER_HI')
        hardLo = pcr.get_colvals(hf, 'MERGED_NET_RATE_APER_LO')
        broadHi = pcr.get_colvals(bf, 'MERGED_NET_RATE_APER_HI')
        broadLo = pcr.get_colvals(bf, 'MERGED_NET_RATE_APER_LO')
    else:
        softHi = pcr.get_colvals(sf, 'NET_RATE_APER_HI')
        softLo = pcr.get_colvals(sf, 'NET_RATE_APER_LO')
        mediumHi = pcr.get_colvals(mf, 'NET_RATE_APER_HI')
        mediumLo = pcr.get_colvals(mf, 'NET_RATE_APER_LO')
        hardHi = pcr.get_colvals(hf, 'NET_RATE_APER_HI')
        hardLo = pcr.get_colvals(hf, 'NET_RATE_APER_LO')
        broadHi = pcr.get_colvals(bf, 'NET_RATE_APER_HI')
        broadLo = pcr.get_colvals(bf, 'NET_RATE_APER_LO')
    raPos = pcr.get_colvals(sf, 'RAPOS')
    decPos = pcr.get_colvals(sf, 'DECPOS')
    component = pcr.get_colvals(sf, 'COMPONENT')
    
    soft = []
    medium = []
    hard = []
    broad = []
    softEr = []
    mediumEr = []
    hardEr = []
    broadEr = []
    
    softCounts = softRate * softTime
    mediumCounts = mediumRate * mediumTime
    hardCounts = hardRate * hardTime
    broadCounts = broadRate * broadTime
    
    for i in range(len(softRate)):
        if broadRate[i]*broadTime[i] >= 10 and softCounts[i] > 0 and mediumCounts[i] > 0 and hardCounts[i] > 0:
            soft.append(softRate[i])
            medium.append (mediumRate[i])
            hard.append(hardRate[i])
            broad.append(broadRate[i])
            softEr.append((softHi[i]-softLo[i])/2)
            mediumEr.append((mediumHi[i]-mediumLo[i])/2)
            hardEr.append((hardHi[i]-hardLo[i])/2)
            broadEr.append((broadHi[i]-broadLo[i])/2)
        else:
            soft.append(1)
            medium.append(1)
            hard.append(1)
            broad.append(1)
            softEr.append(0)
            mediumEr.append(0)
            hardEr.append(0)
            broadEr.append(0)
    
    #print(soft)
    #print(medium)
    #print(hard)
    #print(broad)
    #print(softEr)
    #print(mediumEr)
    #print(hardEr)
    #print(broadEr)
    
    
    softError = (softHi-softLo)/2
    mediumError = (mediumHi-mediumLo)/2
    hardError = (hardHi-hardLo)/2
    broadError = (broadHi-broadLo)/2
    HR1 = np.subtract(medium,soft)/np.add(medium,soft)
    erHR1 = np.divide(2,(np.add(medium,soft))**2) * np.sqrt(np.multiply(np.square(soft),np.square(mediumEr)) + np.multiply(np.square(medium),np.square(softEr)))
    HR2 = np.subtract(hard,medium)/np.add(hard,medium)
    erHR2 = np.divide(2,(np.add(hard,medium))**2) * np.sqrt(np.multiply(np.square(medium),np.square(hardEr)) + np.multiply(np.square(hard),np.square(mediumEr)))
    #print(HR1,erHR1)
    #print(HR2,erHR2)
    
    tbl = make_table_crate(raPos,decPos,component,softCounts,mediumCounts,hardCounts,broadCounts,HR1,erHR1,HR2,erHR2,softRate,softError,mediumRate,mediumError,hardRate,hardError,broadRate,broadError,softTime,mediumTime,hardTime,broadTime,colnames=['RAPOS','DECPOS','COMPONENT','SOFT_COUNTS','MEDIUM_COUNTS','HARD_COUNTS','BROAD_COUNTS','HARDNESS_RATIO_1','HARDNESS_RATIO_1_ERR','HARDNESS_RATIO_2','HARDNESS_RATIO_2_ERR','SOFT_NET_RATE','SOFT_NET_RATE_ERR','MEDIUM_NET_RATE','MEDIUM_NET_RATE_ERR','HARD_NET_RATE','HARD_NET_RATE_ERR','BROAD_NET_RATE','BROAD_NET_RATE_ERR','SOFT_PSF_WEIGHTED_EXPTIME','MEDIUM_PSF_WEIGHTED_EXPTIME','HARD_PSF_WEIGHTED_EXPTIME','BROAD_PSF_WEIGHTED_EXPTIME'])
    raPosU = tbl.get_column('RAPOS')
    raPosU.unit = 'deg'
    decPosU = tbl.get_column('DECPOS')
    decPosU.unit = 'deg'
    softCountsU = tbl.get_column('SOFT_COUNTS')
    softCountsU.unit = 'counts'
    mediumCountsU = tbl.get_column('MEDIUM_COUNTS')
    mediumCountsU.unit = 'counts'
    hardCountsU = tbl.get_column('HARD_COUNTS')
    hardCountsU.unit = 'counts'
    broadCountsU = tbl.get_column('BROAD_COUNTS')
    broadCountsU.unit = 'counts'
    softRateU = tbl.get_column('SOFT_NET_RATE')
    softRateU.unit = 'counts/s'
    softRateErrorU = tbl.get_column('SOFT_NET_RATE_ERR')
    softRateErrorU.unit = 'counts/s'
    hardRateU = tbl.get_column('MEDIUM_NET_RATE')
    hardRateU.unit = 'counts/s'
    hardRateErrorU = tbl.get_column('MEDIUM_NET_RATE_ERR')
    hardRateErrorU.unit = 'counts/s'
    hardRateU = tbl.get_column('HARD_NET_RATE')
    hardRateU.unit = 'counts/s'
    hardRateErrorU = tbl.get_column('HARD_NET_RATE_ERR')
    hardRateErrorU.unit = 'counts/s'
    hardRateU = tbl.get_column('BROAD_NET_RATE')
    hardRateU.unit = 'counts/s'
    hardRateErrorU = tbl.get_column('BROAD_NET_RATE_ERR')
    hardRateErrorU.unit = 'counts/s'
    softTimeU = tbl.get_column('SOFT_PSF_WEIGHTED_EXPTIME')
    softTimeU.unit = 's'
    hardTimeU = tbl.get_column('MEDIUM_PSF_WEIGHTED_EXPTIME')
    hardTimeU.unit = 's'
    hardTimeU = tbl.get_column('HARD_PSF_WEIGHTED_EXPTIME')
    hardTimeU.unit = 's'
    hardTimeU = tbl.get_column('BROAD_PSF_WEIGHTED_EXPTIME')
    hardTimeU.unit = 's'
    pcr.write_file(tbl,"{}/sources/hardness_ratio.fits".format(obj),clobber=True)
    print('FINISHED {}'.format(obj))
    